# 卷积层

## 输入输出的数据格式是： batch * channel * height * width

## 权重格式：output_channels * in_channels * height * width

In [1]:
from mxnet import nd
w=nd.arange(4).reshape((1,1,2,2))
b=nd.array([1])
data=nd.arange(9).reshape((1,1,3,3))
out=nd.Convolution(data,w,b,kernel=w.shape[2:],num_filter=w.shape[1])
print('w: ',w)
print('b: ',b)
print('data: ',data)
print('ou: ',out)

w:  
[[[[0. 1.]
   [2. 3.]]]]
<NDArray 1x1x2x2 @cpu(0)>
b:  
[1.]
<NDArray 1 @cpu(0)>
data:  
[[[[0. 1. 2.]
   [3. 4. 5.]
   [6. 7. 8.]]]]
<NDArray 1x1x3x3 @cpu(0)>
ou:  
[[[[20. 26.]
   [38. 44.]]]]
<NDArray 1x1x2x2 @cpu(0)>


# 窗口移动和边缘填充

In [2]:
out=nd.Convolution(data,w,b,kernel=w.shape[2:],num_filter=w.shape[0 ],stride=(2 ,2),pad=(1,1))
out


[[[[ 1.  9.]
   [22. 44.]]]]
<NDArray 1x1x2x2 @cpu(0)>

In [3]:
w=nd.arange(8).reshape((1,2,2,2))
b=nd.array([1])
data=nd.arange(18).reshape((1,2,3,3))
out=nd.Convolution(data,w,b,kernel=w.shape[2:],num_filter=w.shape[0])
print('w: ',w)
print('b: ',b)
print('data: ',data)
print('out: ',out)
print(w.shape[2:],w.shape[0])

w:  
[[[[0. 1.]
   [2. 3.]]

  [[4. 5.]
   [6. 7.]]]]
<NDArray 1x2x2x2 @cpu(0)>
b:  
[1.]
<NDArray 1 @cpu(0)>
data:  
[[[[ 0.  1.  2.]
   [ 3.  4.  5.]
   [ 6.  7.  8.]]

  [[ 9. 10. 11.]
   [12. 13. 14.]
   [15. 16. 17.]]]]
<NDArray 1x2x3x3 @cpu(0)>
out:  
[[[[269. 297.]
   [353. 381.]]]]
<NDArray 1x1x2x2 @cpu(0)>
(2, 2) 1


In [4]:
w=nd.arange(16).reshape((2,2,2,2))
b=nd.array([1,1])
data=nd.arange(18).reshape((1,2,3,3))
out=nd.Convolution(data,w,b,kernel=w.shape[2:],num_filter=w.shape[0])
print('w: ',w)
print('b: ',b)
print('data: ',data)
print('out: ',out)

w:  
[[[[ 0.  1.]
   [ 2.  3.]]

  [[ 4.  5.]
   [ 6.  7.]]]


 [[[ 8.  9.]
   [10. 11.]]

  [[12. 13.]
   [14. 15.]]]]
<NDArray 2x2x2x2 @cpu(0)>
b:  
[1. 1.]
<NDArray 2 @cpu(0)>
data:  
[[[[ 0.  1.  2.]
   [ 3.  4.  5.]
   [ 6.  7.  8.]]

  [[ 9. 10. 11.]
   [12. 13. 14.]
   [15. 16. 17.]]]]
<NDArray 1x2x3x3 @cpu(0)>
out:  
[[[[ 269.  297.]
   [ 353.  381.]]

  [[ 685.  777.]
   [ 961. 1053.]]]]
<NDArray 1x2x2x2 @cpu(0)>


In [5]:
data=nd.arange(12).reshape((1,2,2,3))
max_pool=nd.Pooling(data,pool_type='max',kernel=(2,2))
avg_pool=nd.Pooling(data,pool_type='avg',kernel=(2,1))
data,max_pool,avg_pool

(
 [[[[ 0.  1.  2.]
    [ 3.  4.  5.]]
 
   [[ 6.  7.  8.]
    [ 9. 10. 11.]]]]
 <NDArray 1x2x2x3 @cpu(0)>,
 
 [[[[ 4.  5.]]
 
   [[10. 11.]]]]
 <NDArray 1x2x1x2 @cpu(0)>,
 
 [[[[1.5 2.5 3.5]]
 
   [[7.5 8.5 9.5]]]]
 <NDArray 1x2x1x3 @cpu(0)>)

# 定义模型

In [6]:
import mxnet as mx 
try:
    ctx=mx.gpu()
    _=nd.zeros((1,),ctx=ctx)
except:
    ctx=mx.cpu()
ctx
    

cpu(0)

In [7]:
weight_scale=.01
num_outputs=10 
w1=nd.random_normal(shape=(20,1,5,5),scale=weight_scale,ctx=ctx)
b1=nd.zeros(w1.shape[0],ctx=ctx)
w2=nd.random_normal(shape=(50,20,3,3),scale=weight_scale,ctx=ctx)
b2=nd.zeros(w2.shape[0],ctx=ctx)

w3=nd.random_normal(shape=(1250,128),scale=weight_scale,ctx=ctx)
b3=nd.zeros(w3.shape[1],ctx=ctx)
w4=nd.random_normal(shape=(w3.shape[1],10),scale=weight_scale,ctx=ctx)
b4=nd.zeros(w4.shape[1],ctx=ctx)
params=[w1,b1,w2,b2,w3,b3,w4,b4]
for param in params:
    param.attach_grad()

In [8]:
def net(X, verbose=False):
    X = X.as_in_context(w1.context)
#     第一层卷积
    h1_conv = nd.Convolution(
        X, w1, b1, kernel=w1.shape[2:], num_filter=w1.shape[0])
    h1_activation = nd.relu(h1_conv1)
    h1 = nd.Pooling(data=h1_activation, pool_type='max',
                    kernel=(2, 2), stride=(2, 2))
#     第一层卷积
    h2_conv = nd.Convolution(
        h1, w2, b2, kernel=w2.shape[2:], num_filter=w2.shape[0])
    h2_activation = nd.relu(h2_conv)
    h2 = nd.Pooling(h2_activation, pool_type='max',
                    kernel=(2, 2), stride=(2, 2))
    h2 = nd.flatten(h2)
#     第一层全连接
    h3_linear = nd.dot(h2, w3)+b3
    h3 = nd.relu(h3_linear)
#     第二层全连接
    h4_linear = nd.dot(h3, w4)+b4
    if verbose:
        print('1st conv block:', h1.shape)
        print('2nd conv block:', h2.shape)
        print('1st dense: ', h3.shape)
        print('2nd dense: ', h4_linear.shape)
        print('output: ', h4_linear)

# 获得数据

In [9]:
from mxnet import gluon
from mxnet import nd
from mxnet import autograd


def transform(data, label):
    return data.astype('float32')/255, label.astype('float32')


minist_train = gluon.data.vision.FashionMNIST(
    root='fashion-mnist/', train=True, transform=transform)
minist_test = gluon.data.vision.FashionMNIST(
    root='fashion-mnist/', train=False, transform=transform)
batch_size = 256
train_data = gluon.data.DataLoader(
    minist_train, batch_size=batch_size, shuffle=True)
test_data = gluon.data.DataLoader(minist_test, batch_size, shuffle=True)

In [10]:
for data ,_ in train_data:
    net(data,True)
    break

MXNetError: Shape inconsistent, Provided = [20,1,5,5], inferred shape=(20,28,5,5)

In [ ]:
def accuracy(output, label):
    return nd.mean(output.argmax(axis=1) == label).asscalar()
def SGD(params,lr):
    for param in params:
        param[:]=param-param.grad * lr
softmax_cross_entropy=gluon.loss.SoftmaxCrossEntropyLoss()
epochs = 5
learning_rate=.01 
for e in range(1, epochs+1):
    train_acc, train_loss = 0., 0.
    for data, label in train_data:
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        SGD(params,learning_rate/batch_size)
        train_acc += accuracy(output, label)
        train_loss += nd.mean(loss).asscalar()
    test_acc = evaluate_accuracy(test_iter, net)
    print('epoch %d,loss:%f..,train_accuracy:%f,test_accuracy:%f..' % (e, train_loss/len(train_iter), train_acc/len(train_iter),
                                                                       test_acc/len(test_iter)))

In [2]:
from mxnet import nd 
p1=nd.array([1,2,4,2,3,])
p2=nd.array([1,2,3,1,1,])
(p1==p2 ).mean() 


[0.4]
<NDArray 1 @cpu(0)>